In [1]:
import pandas as pd
import sqlite3
from Ignore.ignore import FinalDB

In [2]:
con = sqlite3.connect(FinalDB)

In [3]:
df1 = pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/FinalModule_Coursera_V5/data/ChicagoCensusData.csv')
df1.to_sql('CENSUS_DATA',con,if_exists='replace',index=False)
df2 = pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/FinalModule_Coursera_V5/data/ChicagoPublicSchools.csv')
df2.to_sql('CHICAGO_PUBLIC_SCHOOLS',con,if_exists='replace',index=False)
df3 = pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/FinalModule_Coursera_V5/data/ChicagoCrimeData.csv')
df3.to_sql('CHICAGO_CRIME_DATA',con,if_exists='replace',index=False)

533

1. Find the total number of crimes recorded in the CRIME table.

In [4]:
pd.read_sql("""
            SELECT COUNT(*)
            FROM CHICAGO_CRIME_DATA
            ;
            """,con)

,COUNT(*)
0,533


2. List community area names and numbers with per capita income less than 11000.

In [5]:
pd.read_sql("""
            SELECT
                  COMMUNITY_AREA_NAME
                , COMMUNITY_AREA_NUMBER
            FROM
                CENSUS_DATA
            WHERE
                PER_CAPITA_INCOME < 11000
            ;
            """,con)

,COMMUNITY_AREA_NAME,COMMUNITY_AREA_NUMBER
0,West Garfield Park,26.0
1,South Lawndale,30.0
2,Fuller Park,37.0
3,Riverdale,54.0


3. List all case numbers for crimes involving minors?(children are not considered minors for the purposes of crime analysis) 

In [6]:
pd.read_sql("""
            SELECT
                CASE_NUMBER
            FROM
                CHICAGO_CRIME_DATA
            WHERE
                DESCRIPTION LIKE '%MINOR%'
            ;
            """,con)

,CASE_NUMBER
0,HL266884
1,HK238408


4. List all kidnapping crimes involving a child?

In [7]:
pd.read_sql("""
            SELECT
                *
            FROM
                CHICAGO_CRIME_DATA
            WHERE
                    PRIMARY_TYPE = 'KIDNAPPING'
                AND DESCRIPTION LIKE '%CHILD%'
            ;
            """,con)

,ID,CASE_NUMBER,DATE,BLOCK,IUCR,PRIMARY_TYPE,DESCRIPTION,LOCATION_DESCRIPTION,ARREST,DOMESTIC,...,DISTRICT,WARD,COMMUNITY_AREA_NUMBER,FBICODE,X_COORDINATE,Y_COORDINATE,YEAR,LATITUDE,LONGITUDE,LOCATION
0,5276766,HN144152,2007-01-26,050XX W VAN BUREN ST,1792,KIDNAPPING,CHILD ABDUCTION/STRANGER,STREET,0,0,...,15,29.0,25.0,20,1143050.0,1897546.0,2007,41.874908,-87.750249,"(41.874908413, -87.750249307)"


5. List the kind of crimes that were recorded at schools. (No repetitions)

In [8]:
pd.read_sql("""
            SELECT DISTINCT
                  PRIMARY_TYPE
                , DESCRIPTION
            FROM
                CHICAGO_CRIME_DATA
            WHERE
                   LOCATION_DESCRIPTION LIKE '%SCHOOL'
                OR LOCATION_DESCRIPTION LIKE 'SCHOOL%'
                OR LOCATION_DESCRIPTION LIKE '%SCHOOL%'
            ;
            """,con)

,PRIMARY_TYPE,DESCRIPTION
0,BATTERY,SIMPLE
1,BATTERY,PRO EMP HANDS NO/MIN INJURY
2,CRIMINAL DAMAGE,TO VEHICLE
3,NARCOTICS,POSS: HEROIN(WHITE)
4,NARCOTICS,MANU/DEL:CANNABIS 10GM OR LESS
5,ASSAULT,PRO EMP HANDS NO/MIN INJURY
6,CRIMINAL TRESPASS,TO LAND
7,PUBLIC PEACE VIOLATION,BOMB THREAT


6. List the type of schools along with the average safety score for each type.

In [9]:
pd.read_sql("""
            SELECT
                  `Elementary, Middle, or High School` as SCHOOL_TYPE
                , AVG(SAFETY_SCORE)
            FROM 
                CHICAGO_PUBLIC_SCHOOLS
            GROUP BY
                `Elementary, Middle, or High School`
            ;
""",con)

,SCHOOL_TYPE,AVG(SAFETY_SCORE)
0,ES,49.520384
1,HS,49.623529
2,MS,48.000000


7. List 5 community areas with highest % of households below poverty line

In [10]:
pd.read_sql("""
            SELECT
                COMMUNITY_AREA_NAME
            FROM 
                CENSUS_DATA
            ORDER BY
                PERCENT_HOUSEHOLDS_BELOW_POVERTY DESC
            LIMIT 5
            ;
""",con)

,COMMUNITY_AREA_NAME
0,Riverdale
1,Fuller Park
2,Englewood
3,North Lawndale
4,East Garfield Park


8. Which community area is most crime prone? Display the coumminty area number only.

In [11]:
pd.read_sql("""
            SELECT
                COMMUNITY_AREA_NUMBER
            FROM 
                CHICAGO_CRIME_DATA
            GROUP BY
                COMMUNITY_AREA_NUMBER
            ORDER BY
                COUNT(*) DESC
            LIMIT 1
            ;
""",con)

,COMMUNITY_AREA_NUMBER
0,25.0


9. Use a sub-query to find the name of the community area with highest hardship index

In [12]:
pd.read_sql("""
            SELECT
                COMMUNITY_AREA_NAME
            FROM
                CENSUS_DATA
            WHERE
                HARDSHIP_INDEX IN (
                    SELECT
                        MAX(HARDSHIP_INDEX)
                    FROM
                        CENSUS_DATA
                )
            ;
""",con)

,COMMUNITY_AREA_NAME
0,Riverdale


10. Use a sub-query to determine the Community Area Name with most number of crimes?

In [13]:
pd.read_sql("""
            SELECT
                COMMUNITY_AREA_NAME
            FROM
                CENSUS_DATA
            WHERE
                COMMUNITY_AREA_NUMBER IN (
                    SELECT
                        COMMUNITY_AREA_NUMBER
                    FROM 
                        CHICAGO_CRIME_DATA
                    GROUP BY
                        COMMUNITY_AREA_NUMBER
                    ORDER BY
                        COUNT(*) DESC
                    LIMIT 1
                )
                    ;
""",con)

,COMMUNITY_AREA_NAME
0,Austin
